In [ ]:
!pip install matplotlib
!pip install seaborn

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
data = pd.read_csv('all_dt_data.csv')

In [178]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253726 entries, 0 to 253725
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   time_stamp            253726 non-null  object 
 1   rvolt                 253726 non-null  float64
 2   yvolt                 253726 non-null  float64
 3   bvolt                 253726 non-null  float64
 4   rcurrent              253726 non-null  float64
 5   ycurrent              253726 non-null  float64
 6   bcurrent              253726 non-null  float64
 7   real_energy           253726 non-null  float64
 8   apparent              253726 non-null  float64
 9   reactive_lag          253726 non-null  float64
 10  reactive_lead         253726 non-null  float64
 11  pf                    253726 non-null  float64
 12  meter_no              253726 non-null  int64  
 13  yearm                 253726 non-null  int64  
 14  monthm                253726 non-null  int64  
 15  

In [168]:
#group by date
# per_day = df.groupby("datem")["time_stamp","real_energy", "hourm", "number_of_phases"]

def check_day(row):
    #Check the number of phase is 3, then only calculate further.
    if(row["number_of_phases"] != 3):
        return "undefined"
    else:
        if(row['real_energy']!=0 and row['hourm'] in range(6,19)): #day time is consider as 6 am to 6pm 6<=row["hourm"]<=16
           return "day"
        else:
           return "night"

data["day_or_night"] = data.apply(lambda x: check_day(x), axis=1)
#for easy calculation

# # ie. for if the phase number is 3, then only the energy calculated for  day_usage and night-usage
data["day_usage"] = data.apply(lambda x: x["real_energy"] if x["day_or_night"] == "day" else 0, axis=1)
data["night_usage"] = data.apply(lambda x: x["real_energy"] if x["day_or_night"] == "night" else 0, axis=1)

# The energy usage by each day for each DT at day time and night time.
data["day_usage_per_day"] = data.groupby(["datem", "meter_no"])['day_usage'].transform('sum')
data["night_usage_per_day"] = data.groupby(["datem", "meter_no"])['night_usage'].transform('sum')

#check the overall energy usage to verify whether the energy usage is day or night
data["overall_day_or_night"] = data.apply(lambda x: "day" if x["day_usage_per_day"] > x["night_usage_per_day"] else "night", axis=1)
 

In [270]:
def check_phases(row):
    r, b, y = row["rvolt"], row["bvolt"], row["yvolt"]

    if r > 50 and b > 50 and y > 50:
        return 3
    elif (r > 50 and b > 50) or (b > 50 and y > 50) or (r > 50 and y > 50):
        return 2
    elif r > 50 or b > 50 or y> 50:
        return 1
    else:
        return 0

In [325]:
data.to_csv('updated_all_dt_data.csv')

In [274]:
grouped = data.groupby(["datem", "meter_no"])

In [175]:
new_df = pd.DataFrame()

In [321]:
def find_count_phase(x, phase):
   return (x["number_of_phases"] == phase).sum()
    

In [323]:
new_df["date"] = pd.to_datetime(grouped.first()["time_stamp"]).dt.strftime("%d/%m/%Y")
new_df["real_energy"] = grouped["real_energy"].sum()
new_df["day_usage"] = grouped["day_usage"].sum()
new_df["night_usage"] = grouped["night_usage"].sum()
#volatage mean
new_df["r_volt_avg"] = grouped["rvolt"].mean()
new_df["b_volt_avg"] = grouped["bvolt"].mean()
new_df["y_volt_avg"] = grouped["yvolt"].mean()

#volate minumum
new_df["r_volt_min"] = grouped["rvolt"].min()
new_df["b_volt_min"] = grouped["bvolt"].min()
new_df["y_volt_min"] = grouped["yvolt"].min()

#current mean
new_df["r_current_avg"] = grouped["rcurrent"].mean()
new_df["b_current_avg"] = grouped["bcurrent"].mean()
new_df["r_current_avg"] = grouped["ycurrent"].mean()

#current max
new_df["r_current_max"] = grouped["rcurrent"].max()
new_df["b_current_max"] = grouped["bcurrent"].max()
new_df["r_current_max"] = grouped["ycurrent"].max()

#kw load
new_df["kw_load_avg"] = grouped["kw_load"].mean()
new_df["kw_load_max"] = grouped["kw_load"].max()

#Phases count
new_df["phase_1"] = grouped.apply(find_count_phase, 1)
new_df["phase_2"] = grouped.apply(find_count_phase, 2)
new_df["phase_3"] = grouped.apply(find_count_phase, 3)



In [326]:
new_df.to_csv('day_wise_all_dt_data.csv')